In [1]:
from common.tools import Reddit
from common.translations import IO
import pandas as pd
from datetime import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
r = Reddit()
r.subreddits

['SecurityAnalysis', 'StockMarket', 'wallstreetbets', 'wallstreetbets.pd']

In [ ]:
filepath = IO["OUTPUT"]["THREADS"] + subreddit + ".pd"
df = pd.read_pickle(filepath)

# Text pre-processing

In [ ]:
import re
import string

In [ ]:
def cleaning(text):
    text = text.lower()
    text = re.sub("\[.*\]", "", text)
    text = re.sub("\(.*\)", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\w*\d\w*", "", text)
    text = re.sub("\s{2}", " ", text)
    text = text.encode("ascii", "ignore").decode("ascii") # removing emojis
    return text

In [ ]:
df["title"] = df.title.apply(lambda title: cleaning(title))
df["selftext"] = df.selftext.apply(lambda text: cleaning(text))
df["all_comments"] = df.selftext.apply(lambda comments: [cleaning(comment) for comment in comments])

In [ ]:
df.head()

In [ ]:
# TODO split notebook and persist clean data

# Word tokenizing

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
df["title_tokens"] = df.title_cleaned.apply(lambda t: [token for token in word_tokenize(t) if token not in stop_words])

In [ ]:
df.head()

In [ ]:
from nltk.probability import FreqDist

flatten = lambda nested_list: [elm for sublist in nested_list for elm in sublist]

In [ ]:
FreqDist(flatten(df.title_tokens)).plot(30)

## Bigram

In [ ]:
df["bigrams"] = df.title_tokens.apply(lambda t: list(nltk.bigrams(t)))

In [ ]:
df.head()

In [ ]:
FreqDist(flatten(df.bigrams)).plot(30)

## Trigrams

In [ ]:
df["trigrams"] = df.title_tokens.apply(lambda t: list(nltk.trigrams(t)))

In [ ]:
df.head()

In [ ]:
FreqDist(flatten(df.trigrams)).plot(30)

# Part of speech tagging
* shares are NN or JJ (noun or adjective)

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
df["pos"] = df.title_tokens.apply(lambda t: nltk.pos_tag(t))

In [ ]:
df.head()

# Sentiment Analysis
__Output:__
* sentiment score (how positive/negative)
* subjectivity score (how opinionated)

In [ ]:
from textblob import TextBlob

In [ ]:
df["textblob_score"] = df.title.apply(lambda t: TextBlob(t).sentiment)

In [ ]:
df.head()

In [ ]:
freq = FreqDist(df.textblob_score.to_list())
values = list(zip(*freq))
sizes = list(freq.values())
colors = ["r" if v < 0 else "b" for v in values[0]]

In [ ]:
plt.scatter(*values, s=sizes, alpha=.7, c=colors)
plt.xlabel("polarity")
plt.ylabel("subjectivity")
plt.title("TextBlob Sentiment Analysis Score")

# Topic Modeling LDA (Latent Dirichlet Allocation)

In [1]:
from gensim import matutils, models

AttributeError: module 'numpy.random' has no attribute 'default_rng'